In [1]:
import openmm as mm
import numpy as np
import openmm.app as app
import simtk.unit as unit
from openmmforcefields.generators import GAFFTemplateGenerator as gen
from openff.toolkit import Topology
import pytraj as pt
import os
import matplotlib.pyplot as plt

from openmm.app import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet



In [2]:
base = '/scratch/htc/fsafarov/structures/7rqz_sep_2025/7qrz'

#We use psf and crd system because pdb files does not support the 6-digit base system for ATOM ID's, which is in our case utilized for POPC. 
#Input below is from psf and crd from CHARMM-GUI.
psf = CharmmPsfFile(os.path.join(base, 'step5_assembly.psf'))
crd = CharmmCrdFile(os.path.join(base, 'step5_assembly.crd'))



In [3]:
state = mm.XmlSerializer.deserialize(open('/scratch/htc/fsafarov/states/trpv1/7rqz/relaxation.xml').read())
centered_positions = state.getPositions()

In [4]:
with open('/scratch/htc/fsafarov/states/trpv1/7rqz/relaxed_system.xml') as input:
    system = mm.XmlSerializer.deserialize(input.read())

In [5]:
integrator_pr = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)

In [6]:
platform = mm.Platform.getPlatformByName('CUDA')
# #to run the simulation on several GPU's parallelly
properties  = {'CudaDeviceIndex': '0,1,2', 'CudaPrecision': 'mixed'}

In [7]:
simulation_pr = app.Simulation(psf.topology, system, integrator_pr, platform, properties)
simulation_pr.loadCheckpoint('/scratch/htc/fsafarov/traj/traj_trp/mor_amber_protocol_2/7rqz/step6.chk')
state_1 = simulation_pr.context.getState(getVelocities=True)
velocities = state_1.getVelocities(asNumpy = True)

In [8]:
for i in range(system.getNumForces()):
    force = system.getForce(i)
    if isinstance(force, mm.MonteCarloBarostat):
        print('fuck you')
        system.removeForce(i)

In [9]:
for i in range(system.getNumForces()):
    force = system.getForce(i)
    print(force)

<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f84f8d30d20> >
<openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f84f4838990> >
<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f84f8d30b70> >
<openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7f84f48389c0> >
<openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x7f850ecafb10> >
<openmm.openmm.CMAPTorsionForce; proxy of <Swig Object of type 'OpenMM::CMAPTorsionForce *' at 0x7f84f8d30b70> >
<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f84f8d30d20> >
<openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7f84f8d30b70> >
<openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::C

In [10]:
system.addForce(mm.MonteCarloAnisotropicBarostat(
    mm.Vec3(1, 1, 1)*unit.atmosphere,   
    310.15*unit.kelvin,
    False, False, True            # scaleX, scaleY, scaleZ
))

11

In [11]:
simulation_pr.context.reinitialize()
simulation_pr.context.setPositions(centered_positions)
simulation_pr.context.setVelocities(velocities)

In [12]:
a_vec, b_vec, c_vec = state.getPeriodicBoxVectors()


simulation_pr.context.setPeriodicBoxVectors(a_vec,b_vec,c_vec)

In [13]:
simulation_pr.context.setParameter('k_1', 0.0*unit.kilocalories_per_mole/unit.angstrom**2)
simulation_pr.context.setParameter('k_2', 0.0*unit.kilocalories_per_mole/unit.angstrom**2)

In [14]:
dict(simulation_pr.context.getParameters()) #check the padrameters!!!

{'AnisotropicMonteCarloTemperature': 310.15,
 'MonteCarloPressureX': 1.01325,
 'MonteCarloPressureY': 1.01325,
 'MonteCarloPressureZ': 1.01325,
 'k_1': 0.0,
 'k_2': 0.0}

In [15]:
#check the forces
state_2 = simulation_pr.context.getState(getForces=True, getEnergy=True)
forces = state_2.getForces(asNumpy = True).value_in_unit(unit.kilojoule/unit.nanometer/unit.mole)
norm = np.linalg.norm(forces, axis = 1)
max_force = max(norm)
np.argmax(norm), np.max(norm)

(50287, 2498.315497144909)

In [16]:
#check energy
print("Potential energy:", state_2.getPotentialEnergy())
print("Kinetic energy:", state_2.getKineticEnergy())

Potential energy: -1481581.807650336 kJ/mol
Kinetic energy: 339620.3675430586 kJ/mol


In [17]:
simulation_pr.reporters.append(app.DCDReporter('/scratch/htc/fsafarov/traj/traj_trp/mor_amber_protocol_2/7rqz/production_1.dcd', 100000, enforcePeriodicBox=True))
simulation_pr.reporters.append(app.CheckpointReporter('/scratch/htc/fsafarov/traj/traj_trp/mor_amber_protocol_2/7rqz/production_1.chk', 100000))
simulation_pr.step(50000000)


In [18]:
#check the forces
state_3 = simulation_pr.context.getState(getForces=True, getEnergy=True)
forces = state_3.getForces(asNumpy = True).value_in_unit(unit.kilojoule/unit.nanometer/unit.mole)
norm = np.linalg.norm(forces, axis = 1)
max_force = max(norm)
np.argmax(norm), np.max(norm)

(27815, 5108.161194864958)

In [ ]:
simulation_pr.saveState('/scratch/htc/fsafarov/states/mor_amber_protocol_2/production_3.xml')

In [ ]:
with open('/scratch/htc/fsafarov/states/mor_amber_protocol_2/prod_system_3.xml', 'w') as f:
    f.write(mm.XmlSerializer.serialize(system))